In [ ]:
# collect lancet data
import sys
#import icd
import df_utils
#import utils
import pandas as pd

'''
Age cataract diagnosed: 4700
Age when diabetes-related eye disease diagnosed - 5901

LDL - 30780

Head Injury - ICD codes S00-S09

Physical inactivity - MET minutes per week for moderate activity - 22038

Smoking - Pack years of smoking - 20161

Excessive alcohol consumption - 1558

Hypertension
Source of report of primary/secondary hypertension: 131287/131295
Date ^ reported: 131286/131294
Source for gestational with/without proteinuria: 132189/132187
Date ^ reported: 132188/132186
Source for pre-existing hypertension complicating pregnancy, childbirth and the puerperium: 132181
Date ^ reported: 132180

Obesity - Date E66 first reported: 130792; Source: 130793

Diabetes (Date first reported; Source)
Insulin-dependent: 130706; 130707
Non-insulin-dependent: 130708; 130709
Malnutrition-related: 130710; 130711
Other specified: 130712; 130713
Unspecified: 130714; 130715
During pregnancy: 132202; 132203

Hearing loss
Currently suffering: 28627
Length of time suffering from hearing loss: 28628
Extent affected by hearing loss: 28629
Date first reported; Source:
Conductive and sensorineural: 131258; 131259
Other hearing loss: 131260; 131261

Depression (Date first reported; Source)
Depressive episode: 130894; 130895
Recurrent: 130896; 130897

Infrequent social contact:
Frequency of friends/family visit: 1031
Loneliness, isolation: 2020

Air pollution
Inverse distance to nearest major road: 24012
Nitrogen dioxide air pollution, 2007: 24018
Particulate matter air pollution (pm10), 2007: 24019
Particulate matter air pollution (pm2.5), 2010: 24006
Sum of read length of major roads within 100m: 24015
Traffic intensity on nearest major road: 24011

NOTE: we don't have 28627, 28628, 28629
'''

ids_of_interest = [4700, 5901, 30780, 22038, 20161, 1558, 131287, 131295, 131286, 131294, 132189, 132187, 132188, 132186, 132181, 132180, 130792, 130793,
             130706, 130707, 130708, 130709, 130710, 130711, 130712, 130713, 130714, 130715, 132202, 132203, 28627, 28628, 28629, 131258, 131259, 131260, 131261,
             130894, 130895, 130896, 130897, 1031, 2020, 24012, 24018, 24019, 24006, 24015, 24011]
ids_of_interest = [str(x) for x in ids_of_interest]

df1 = pd.read_csv('../../../uk_biobank/project_52887_676883/ukb676883.csv', nrows=1)
filtered_columns = ['eid'] + [col for col in df1.columns if col.split('-')[0] in ids_of_interest]
df1 = pd.read_csv('../../../uk_biobank/project_52887_676883/ukb676883.csv', usecols=filtered_columns)

df2 = pd.read_csv('../../../uk_biobank/project_52887_669338/ukb669338.csv', nrows=1)
filtered_columns = ['eid'] + [col for col in df2.columns if col.split('-')[0] in ids_of_interest]
df2 = pd.read_csv('../../../uk_biobank/project_52887_669338/ukb669338.csv', usecols=filtered_columns)
ins0_col = ['eid'] + [x for x in df2.columns if '-0' in x]
df2 = df2.loc[:, ins0_col]



In [ ]:
def _starts_with_any(value, start_strings):
    if value is None:
        return False
    return any(str(value).startswith(s) for s in start_strings)

def pull_icds(fieldid_icd, fieldid_date, code_icd):

    '''
    fieldid_icd: list - either 41270 (ICD10) or 41271 (ICD9) (diagnoses)
    fieldid_date: list - either 41280 (ICD10) or 41281 (ICD9) (Date of first in-patient diagnosis)
    code_icd: list - ICD codes you want to pull
    '''

    # Get the directory where the script is located
    file_path = "/n/groups/patel/randy/proj_idp/tidy_data/icd9_icd10.parquet"

    df = pd.read_parquet(file_path)
    # Subset columns for either ICD9 or ICD10
    icd_matching_columns = [col
                            for col in df.columns
                            if any(col.startswith(s)
                                   for s in fieldid_icd)]

    icd_date_columns = [col
                        for col in df.columns
                        if any(col.startswith(s)
                               for s in fieldid_date)]

    # Apply the filtering
    match_df = df[df[icd_matching_columns].map(
                lambda x: _starts_with_any(x, code_icd)).any(axis=1)]

    icd_df = match_df.loc[:, ['eid'] + icd_matching_columns].reset_index(
                                                            drop=True)
    icd_date_df = match_df.loc[:, ['eid'] + icd_date_columns].reset_index(
                                                            drop=True)

    return icd_df, icd_date_df

def remove_participants_full_missing(df, columns_to_check=None):

    if columns_to_check is None:
        columns_to_check = [col for col in df.columns.tolist() if col != 'eid']
    
    df_sub = df.loc[:, columns_to_check]
    
    na_counts = df_sub.isna().sum(axis=1)

    keep = na_counts[na_counts < len(columns_to_check)]

    df_keep = df.iloc[keep, :]

    return df_keep

def _find_earliest_dates(icd_df, icd_date_df, code_icd):
    """
    Finds the earliest date for each row where the ICD code is present.
    
    icd_df: DataFrame containing ICD codes
    icd_date_df: DataFrame containing dates corresponding to the ICD codes
    code_icd: list of ICD codes to search for
    
    Returns:
    DataFrame with earliest dates for each row where the ICD code is present.
    """
    earliest_dates = []

    for i, row in icd_df.iterrows():
        indices = [j for j, col in enumerate(icd_df.columns[1:]) if any(str(row[col]).startswith(code) for code in code_icd)]
        if indices:
            dates = [pd.to_datetime(icd_date_df.iloc[i, j + 1], errors='coerce') for j in indices]
            earliest_date = min(dates)
        else:
            earliest_date = pd.NaT
        earliest_dates.append(earliest_date)

    icd_df['earliest_date'] = earliest_dates
    return icd_df[['eid', 'earliest_date']]
    
def pull_earliest_dates(fieldid_icd, fieldid_date, code_icd):
    icd_df, icd_date_df = pull_icds(fieldid_icd, fieldid_date, code_icd)
    earliest_dates = _find_earliest_dates(icd_df, icd_date_df, code_icd)
    return earliest_dates


In [ ]:
icd_df, icd_date_df = pull_icds(['41270'], ['41280'], ['F32'])

df1 = remove_participants_full_missing(df1)
df2 = remove_participants_full_missing(df2)
df3 = df1.merge(df2, how='outer', on='eid')

# Head injury ICD codes don't have pre-made source/date columns
icd = icd_df.merge(icd_date_df, on='eid', how='outer')

lancet_vars = df3
depression_icd = icd
depression_date = icd_date_df


In [ ]:
icd_df, icd_date_df = pull_icds(['41270'], ['41280'], ['G30'])

df1 = remove_participants_full_missing(df1)
df2 = remove_participants_full_missing(df2)
df3 = df1.merge(df2, how='outer', on='eid')

# Head injury ICD codes don't have pre-made source/date columns
icd = icd_df.merge(icd_date_df, on='eid', how='outer')

lancet_vars = df3
AD_icd = icd
AD_date = icd_date_df


In [ ]:
# see date format of 
import ukb_utils

hypertension_cols = ['131287-0.0', '131295-0.0', '132189-0.0', '132187-0.0', '132181-0.0']
hypertension_dates = ['131286-0.0', '131294-0.0', '132188-0.0', '132186-0.0', '132180-0.0']
lancet_vars[hypertension_dates]

lancet_vars[hypertension_dates] = lancet_vars[hypertension_dates].apply(pd.to_datetime, errors='coerce')

lancet_vars = ukb_utils.binary_encode_column_membership_datacoding2171(lancet_vars, hypertension_cols, 'hypertension')
lancet_vars['hypertension_dx'] = lancet_vars[hypertension_dates].min(axis=1)

In [ ]:
import ukb_utils

# Age cataract diagnosed and Age when diabetes-related eye disease diagnosed are NA or Age. Transform so that NA=0 and non-NA=1
lancet_vars['cataract'] = lancet_vars['4700-0.0'].notna().astype(int)
lancet_vars['diabetic_retinopathy'] = lancet_vars['5901-0.0'].notna().astype(int)

# head injury column where the value is 0 if the eid is not in the eid column of head_injury_icd, and 1 if it is
lancet_vars['head_injury'] = lancet_vars.eid.isin(head_injury_icd.eid).astype(int)
lancet_vars['head_injury_dx'] = lancet_vars.eid.isin(head_injury_date.eid)
 
# ordinal encoding for alcohol intake frequency 
# Define the correct order for the ordinal variable
correct_order = [-3, 6, 5, 4, 3, 2, 1]
lancet_vars['alcohol_consumption'] = pd.Categorical(lancet_vars['1558-0.0'], categories=correct_order, ordered=True)
# Convert the ordered categorical data to integer codes
lancet_vars['alcohol_consumption'] = lancet_vars['alcohol_consumption'].cat.codes

# hypertension
hypertension_cols = ['131287-0.0', '131295-0.0', '132189-0.0', '132187-0.0', '132181-0.0']
hypertension_dates = ['131286-0.0', '131294-0.0', '132188-0.0', '132186-0.0', '132180-0.0']

lancet_vars[hypertension_dates] = lancet_vars[hypertension_dates].apply(pd.to_datetime, errors='coerce')

lancet_vars = ukb_utils.binary_encode_column_membership_datacoding2171(lancet_vars, hypertension_cols, 'hypertension')
lancet_vars['hypertension_dx'] = lancet_vars[hypertension_dates].min(axis=1)

# obesity 
lancet_vars = ukb_utils.binary_encode_column_membership_datacoding2171(lancet_vars, ['130793-0.0'], 'obesity')

lancet_vars['obesity_dx'] = lancet_vars['130792-0.0']

# diabetes
diabetes_cols = ['130707-0.0', '130709-0.0', '130711-0.0', '130713-0.0', '130715-0.0', '132203-0.0']
diabetes_dates = ['130706-0.0', '130708-0.0', '130710-0.0', '130712-0.0', '130713-0.0', '132202-0.0']
lancet_vars[diabetes_dates] = lancet_vars[diabetes_dates].apply(pd.to_datetime, errors='coerce')

lancet_vars = ukb_utils.binary_encode_column_membership_datacoding2171(lancet_vars, diabetes_cols, 'diabetes')
lancet_vars['diabetes_dx'] = lancet_vars[diabetes_dates].min(axis=1)

# hearing loss
hearing_cols = ['131259-0.0', '131261-0.0']
hearing_dates = ['130894-0.0', '130896-0.0']
lancet_vars[hearing_dates] = lancet_vars[hearing_dates].apply(pd.to_datetime, errors='coerce')

lancet_vars = ukb_utils.binary_encode_column_membership_datacoding2171(lancet_vars, hearing_cols, 'hearing_loss')
lancet_vars['hearing_loss_dx'] = lancet_vars[hearing_dates].min(axis=1)

# depression
depression_cols = ['130895-0.0', '130897-0.0']
depression_dates = ['130894-0.0', '130896-0.0']
lancet_vars[depression_dates] = lancet_vars[depression_dates].apply(pd.to_datetime, errors='coerce')

lancet_vars = ukb_utils.binary_encode_column_membership_datacoding2171(lancet_vars, depression_cols, 'depression')
lancet_vars['depression_dx'] = lancet_vars[depression_dates].min(axis=1)

# ordinal encoding for frequency of friends/family visit
correct_order = [-3, -1, 7, 6, 5, 4, 3, 2, 1]
lancet_vars['freq_friends_family_visit'] = pd.Categorical(lancet_vars['1031-0.0'], categories=correct_order, ordered=True)
# Convert the ordered categorical data to integer codes
lancet_vars['freq_friends_family_visit'] = lancet_vars['freq_friends_family_visit'].cat.codes

lancet_vars.to_parquet('./snp_parquets/lancet2024_preprocessed.parquet')

Pulling Diagnosis Dates


In [ ]:
import pandas as pd
# dates

# Pull and rename each diagnosis date
depression_dx = pull_earliest_dates(['41270'], ['41280'], ['F32']).rename(columns={'earliest_date': 'depression_dx'})
ad_dx = pull_earliest_dates(['41270'], ['41280'], ['G30']).rename(columns={'earliest_date': 'ad_dx'})
hypertension_dx = pull_earliest_dates(['41270'], ['41280'], ['I10']).rename(columns={'earliest_date': 'hypertension_dx'})
diabetes_dx = pull_earliest_dates(['41270'], ['41280'], ['E08', 'E09', 'E10', 'E11', 'E12', 'E13']).rename(columns={'earliest_date': 'diabetes_dx'})
obesity_dx = pull_earliest_dates(['41270'], ['41280'], ['E66']).rename(columns={'earliest_date': 'obesity_dx'})

# Merge all on 'eid'
merged_dx = depression_dx \
    .merge(ad_dx, on='eid', how='outer') \
    .merge(hypertension_dx, on='eid', how='outer') \
    .merge(diabetes_dx, on='eid', how='outer') \
    .merge(obesity_dx, on='eid', how='outer')



In [ ]:
cols_to_check = [col for col in merged_dx.columns if col != 'eid']
mask = merged_dx[cols_to_check].isna().all(axis=1)

filtered_dx = merged_dx[~mask]

In [ ]:
filtered_dx

In [ ]:
import pandas as pd
df = pd.read_parquet('../../../randy/proj_idp/tidy_data/prs_Alz/prs_Alz.parquet', engine = 'fastparquet')

In [ ]:
df

In [ ]:
filtered_dx.to_parquet('lancet_dx_dates.parquet')

In [ ]:
depression_treatment['medicated'] = np.where(
    depression_treatment['29039-0.0'].isna(),
    np.nan,
    np.where(depression_treatment['29039-0.0'].isin(range(1, 8)), 1, 0)
)

In [ ]:
depression_treatment

In [ ]:
if_depression_treatment = pd.read_csv('../../../uk_biobank/project_52887_676883/ukb676883.csv', usecols=['eid', '29038-0.0'])

In [ ]:
medications = pd.read_csv('../../../uk_biobank/project_52887_676883/ukb676883.csv', usecols=['eid', '29039-0.0'])

In [ ]:
if_depression_treatment['29038-0.0'].value_counts()

In [ ]:
import numpy as np

# take all the one values in "medicated", and add then to the "if_depression" column

medicated_col = if_depression_treatment['29038-0.0']

if_depression_treatment['depression_treated?'] = np.where(
    medicated_col.isna(),
    np.nan,
    np.where(medicated_col.isin([1,2]), medicated_col, 0)
)

In [ ]:
if_depression_treatment['depression_treated?'].value_counts()

In [ ]:
medicated_col = medications['29039-0.0']

medications['medicated'] = np.where(
    medicated_col.isna(),
    np.nan,
    np.where(medicated_col.isin(range(1, 8)), 1, 0)
)

In [ ]:
medications['medicated'].value_counts()

In [ ]:
medications['medicated'].value_counts()

In [ ]:
depression_treatment = if_depression_treatment.merge(medications, on = 'eid')

In [ ]:
depression_treatment

In [ ]:
if_depression_treatment = if_depression_treatment.drop(columns = ['29038-0.0']).dropna()

In [ ]:
if_depression_treatment.to_parquet('depression_treatment.parquet')

Cognitive Testing and Depression Treatment

In [ ]:
import pickle
with open('../../../randy/rfb/tidy_data/UKBiobank/dementia/cognitive_tests/cognitive_columns.pkl', 'rb') as f:
    cognitive_tests = pickle.load(f)

cognitive_tests_df = pd.read_csv('../../../uk_biobank/project_52887_676883/ukb676883.csv', usecols=['eid'] + cognitive_tests)

# remove rows with all NaNs
mask = cognitive_tests_df.drop("eid", axis=1).isna().all(axis=1)
cleaned_ct = cognitive_tests_df[~mask]

cleaned_ct.to_parquet('cognitive_test_results.parquet', engine = 'fastparquet')
